In [191]:
import numpy as np
import pandas as pd
import cv2

det = pd.read_csv('det/det.txt', sep=',', header=None)

columns = [
    "frame",
    "id",
    "bb_left",
    "bb_top",
    "bb_width",
    "bb_height",
    "conf",
    "x",
    "y",
    "z"
]
det.columns = columns
og_len = det["frame"].max()
det

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.567,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.439,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.601,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.013,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.949,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
4897,525,-1,1479,580,28.302,64.224,36.629,-1,-1,-1
4898,525,-1,1806,521,36.723,83.333,23.327,-1,-1,-1
4899,525,-1,1820,381,123.420,280.060,14.567,-1,-1,-1
4900,525,-1,1655,425,94.660,214.810,12.803,-1,-1,-1


In [192]:
# filter bboxes with conf
conf_threshold = 30.0
det = det[det.conf >= conf_threshold]
det

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
1918,225,-1,682,275,250.000,567.31,104.360,-1,-1,-1
2182,247,-1,969,283,226.740,514.53,107.300,-1,-1,-1
2344,261,-1,760,353,207.450,470.74,104.100,-1,-1,-1
2358,262,-1,760,353,207.450,470.74,101.810,-1,-1,-1
2545,278,-1,1112,353,207.450,470.74,99.887,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
4072,441,-1,1726,415,103.170,234.13,107.460,-1,-1,-1
4082,442,-1,1726,415,103.170,234.13,100.230,-1,-1,-1
4105,444,-1,1742,415,103.170,234.13,105.790,-1,-1,-1
4118,445,-1,1742,415,103.170,234.13,99.857,-1,-1,-1


In [193]:
det.drop(columns=["x", "y", "z"], inplace=True)

C:\Users\Tom\AppData\Local\Temp\ipykernel_15644\1947031159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det.drop(columns=["x", "y", "z"], inplace=True)


In [194]:
def get_frames(df):
    frames = {}
    for frame in df["frame"].unique():
        current = []
        for _, row in df[df["frame"] == frame].iterrows():
            current.append(row[2:6].values)
        frames[frame] = current
    return frames

det_frames = get_frames(det)
det_frames[list(det_frames.keys())[0]]

[array([682.  , 275.  , 250.  , 567.31])]

In [195]:
# compute jaccard index
def jaccard_index(a, b):
    a_left, a_top, a_width, a_height = a
    b_left, b_top, b_width, b_height = b
    a_right = a_left + a_width
    a_bottom = a_top + a_height
    b_right = b_left + b_width
    b_bottom = b_top + b_height
    xA = max(a_left, b_left)
    yA = max(a_top, b_top)
    xB = min(a_right, b_right)
    yB = min(a_bottom, b_bottom)
    interArea = max(0, xB - xA) * max(0, yB - yA)
    aArea = a_width * a_height
    bArea = b_width * b_height
    return interArea / float(aArea + bArea - interArea)

In [196]:
# compute jaccard index for all frames between frame n and frame n - 1
def jaccard_index_frames(det_frames):
    jaccard_index_frames = {}
    frame_indices = list(det_frames.keys())
    for frame_index, frame in enumerate(det_frames):
        if frame == 1:
            continue
        jaccard_index_frame = np.zeros((len(det_frames[frame]), len(det_frames[frame_indices[frame_index - 1]])))
        for i, a in enumerate(det_frames[frame]):
            for j, b in enumerate(det_frames[frame_indices[frame_index - 1]]):
                jaccard_index_frame[i][j] = jaccard_index(a, b)
        jaccard_index_frames[frame] = jaccard_index_frame
    return jaccard_index_frames

jaccard_index_frames = jaccard_index_frames(det_frames)
jaccard_index_frames[list(jaccard_index_frames.keys())[0]]

array([[0.]])

In [197]:
# associate the detections to tracks in a greedy manner using IoU threshold sigma_iou
sigma_iou = 0.7
def associate_detections_to_tracks(jaccard_index_frames):
    tracks = {}
    jaccard_values = {}
    for frame in jaccard_index_frames.keys():
        tracks[frame] = []
        jaccard_values[frame] = []
        for jaccard_index_frame in jaccard_index_frames[frame]:
            if len(jaccard_index_frame) == 0:
                tracks[frame].append(-1)
                continue
            max_index = np.argmax(jaccard_index_frame)
            if jaccard_index_frame[max_index] >= sigma_iou:
                tracks[frame].append(max_index)
                jaccard_values[frame].append(jaccard_index_frame[max_index])
            else:
                tracks[frame].append(-1)
                jaccard_values[frame].append(-1)
    return tracks, jaccard_values

tracks, jaccard_values = associate_detections_to_tracks(jaccard_index_frames)

In [198]:
# track management
def track_management(tracks, jaccard_values):
    """
    Track management
     Each object can be assigned to only one trajectory (ID)
     Create and update lists for matches, unmatched detections and unmatched tracks
     Matches: IoU ≥ sigma_iou -> matched track
     Unmatched tracks -> delete track
     Unmatched detection -> create new tracks
    """
    bboxes_for_each_frame = {}
    frame_indices = list(tracks.keys())
    for frame_index, current_frame in enumerate(frame_indices[1:]):
        previous_frame = frame_indices[frame_index - 1]
        p_tracks = np.array(tracks[previous_frame])
        c_tracks = np.array(tracks[current_frame])
        # sort
        jaccard_dict = {}
        for i, t in enumerate(c_tracks):
            jaccard_dict[t] = jaccard_values[current_frame][i]
        p_tracks = np.sort(p_tracks)
        c_tracks = np.sort(c_tracks)
        # remove -1
        p_tracks = p_tracks[p_tracks >= 0]
        c_tracks = c_tracks[c_tracks >= 0]
        # uniques only
        p_tracks = np.unique(p_tracks)
        c_tracks = np.unique(c_tracks)
        # print("----------------------")
        # print(f"Previous Tracks Frame {previous_frame}: {p_tracks}")
        # print(f"Current Tracks Frame {current_frame}: {c_tracks}")
        matches = []
        unmatched_tracks = [] # objects that have disappeared
        unmatched_detections = [] # new appearances of objects

        for i, track in enumerate(p_tracks):
            if track not in c_tracks:
                unmatched_tracks.append(track)

        for i, track in enumerate(c_tracks):
            if track in p_tracks:
                matches.append(track)
            else:
                unmatched_detections.append(track)
        # print(f"Matches Frame {current_frame}: {matches}")
        # print(f"Unmatched Tracks Frame {current_frame}: {unmatched_tracks}")
        # print(f"Unmatched Detections Frame {current_frame}: {unmatched_detections}")
        bboxes_for_each_frame[current_frame] = {
            "matches": matches,
            "unmatched_tracks": unmatched_tracks,
            "unmatched_detections": unmatched_detections,
            "jaccard_dict": jaccard_dict
        }
    return bboxes_for_each_frame

bboxes_for_each_frame = track_management(tracks, jaccard_values)
bboxes_for_each_frame

{247: {'matches': [],
  'unmatched_tracks': [],
  'unmatched_detections': [],
  'jaccard_dict': {-1: -1}},
 261: {'matches': [],
  'unmatched_tracks': [],
  'unmatched_detections': [],
  'jaccard_dict': {-1: -1}},
 262: {'matches': [],
  'unmatched_tracks': [],
  'unmatched_detections': [0],
  'jaccard_dict': {0: 1.0000000000000007}},
 278: {'matches': [],
  'unmatched_tracks': [],
  'unmatched_detections': [],
  'jaccard_dict': {-1: -1}},
 279: {'matches': [0],
  'unmatched_tracks': [],
  'unmatched_detections': [],
  'jaccard_dict': {0: 1.0000000000000007}},
 280: {'matches': [],
  'unmatched_tracks': [],
  'unmatched_detections': [0],
  'jaccard_dict': {0: 0.7327208185424937}},
 281: {'matches': [0],
  'unmatched_tracks': [],
  'unmatched_detections': [],
  'jaccard_dict': {0: 1.0000000000000007}},
 282: {'matches': [0],
  'unmatched_tracks': [],
  'unmatched_detections': [],
  'jaccard_dict': {0: 1.0000000000000007}},
 283: {'matches': [0],
  'unmatched_tracks': [],
  'unmatched_de

In [199]:
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

"""
Develop an interface for tracking results check to see if the tracker properly keeps track of objects by
associating the correct IDs in the video stream
 Draw rectangular bounding box around the detected object in images
 Draw attributed ID to each tracked objects
 Draw the trajectory (tracking path ) in an image
"""

def draw_bboxes(frame, bboxes, color):
    for bbox in bboxes:
        x, y, w, h = bbox.astype(int)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

def draw_ids(frame, bboxes, jaccard_dict, ids, color):
    for bbox, id in zip(bboxes, ids):
        x, y, w, h = bbox.astype(int)
        cv2.putText(frame, str(id), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)
        if jaccard_dict is not None:
            jac = jaccard_dict[id]
            cv2.putText(frame, str(round(jac, 2)), (x, y + h + 10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 2)

def draw_trajectories(frame, centroids, previous_centroids, ids, color):
    for bbox, id in zip(centroids, ids):
        # draw line between previous centroid and current centroid
        if id > len(previous_centroids) - 1:
            continue
        previous_centroid = previous_centroids[id]
        previous_centroid = (int(previous_centroid[0]), int(previous_centroid[1]))
        current_centroid = (int(bbox[0]), int(bbox[1]))
        cv2.line(frame, previous_centroid, current_centroid, color, 2)

frame_ids = list(bboxes_for_each_frame.keys())

centroids = {}
frame_imgs = []
frame_index = 0
for frame in tqdm(range(1, og_len + 1)):
    img = cv2.imread(f"img1/{frame:06d}.jpg")
    if frame not in frame_ids:
        frame_imgs.append(img)
        continue
    frame_index += 1
    matches = bboxes_for_each_frame[frame]["matches"]
    unmatched_detections = bboxes_for_each_frame[frame]["unmatched_detections"]
    unmatched_tracks = bboxes_for_each_frame[frame]["unmatched_tracks"]
    jaccard_values = bboxes_for_each_frame[frame]["jaccard_dict"]

    bbox_matches = []
    bbox_unmatched_detections = []
    bbox_unmatched_tracks = []
    for i, bbox in enumerate(det_frames[frame]):
        if i in matches:
            bbox_matches.append(bbox)
        elif i in unmatched_detections:
            bbox_unmatched_detections.append(bbox)
        elif i in unmatched_tracks:
            bbox_unmatched_tracks.append(bbox)
    
    centroids[frame] = []
    for bb in bbox_matches: 
        centroids[frame].append((bb[0] + bb[2] // 2, bb[1] + bb[3] // 2))
    
    draw_bboxes(img, bbox_matches, (255, 255, 255))
    draw_ids(img, bbox_matches, jaccard_values, matches, (255, 255, 255))

    draw_bboxes(img, bbox_unmatched_detections, (0, 255, 0))
    draw_ids(img, bbox_unmatched_detections, jaccard_values, unmatched_detections, (0, 255, 0))

    draw_bboxes(img, bbox_unmatched_tracks, (0, 0, 255))
    draw_ids(img, bbox_unmatched_tracks, None, unmatched_tracks, (0, 0, 255))
    
    if frame_index > frame_ids[0]:
        previous_frame_index = frame_ids[frame_index - 1]
        draw_trajectories(img, centroids[frame], centroids[previous_frame_index], matches, (255, 255, 255))

    frame_imgs.append(img)

100%|██████████| 525/525 [00:08<00:00, 59.94it/s]


In [200]:
# pop a window with a video stream from the frame_imgs
fps = 30
def show_video(frame_imgs):
    for frame in frame_imgs:
        cv2.imshow("video", frame)
        if cv2.waitKey(1000 // fps) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

show_video(frame_imgs)